In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import statsmodels.formula.api as smf
from yahoo_fin import stock_info as si

In [11]:
# Pulls daily adjusted close prices for one ticker. Returns: date, price, ticker
def get_daily_prices(ticker, start_date, end_date):
    df = si.get_data(ticker, start_date=start_date, end_date=end_date).reset_index()
    
    df["price"] = df["adjclose"]
    df["ticker"] = ticker

    return df


def get_prices(tickerlist, start, end):
    df = pd.DataFrame(columns=["date", "price", "ticker"])

    for ticker in tickerlist:
        df = pd.concat(
            [df, get_daily_prices(ticker, start, end)],
            ignore_index=True
        )

    return df


def run_diff_in_diff(df, event_date, treated):
    start_date = event_date - datetime.timedelta(days=180)

    df = df[(df['date'] >= start_date) & (df['date'] <= event_date)]

    df['treated'] = np.where(df['ticker'] == treated, 1, 0)
    df['post'] = np.where(df['date'] >=  event_date, 1, 0)
    df['treated_post'] = df['treated'] * df['post']

    formula = "returns ~ treated + post + treated_post"
    model = smf.ols(formula=formula, data=df).fit()

    return model


def difference_in_trends (df, event_date, treated):
    start_date = event_date - datetime.timedelta(days=180)

    df = df[(df['date'] >= start_date) & (df['date'] <= event_date)]

    df['treated'] = np.where(df['ticker'] == treated, 1, 0)
    df['trend'] = (event_date - df['date']).dt.days
    df['treated_trend'] = df['treated'] * df['trend']
    
    formula = "returns ~ treated + trend + treated_trend"
    model = smf.ols(formula=formula, data=df).fit()

    return model